In [13]:
from bs4 import BeautifulSoup
import urllib3
from urllib.parse import urljoin
import re
import nltk

# Crawler das urls

In [ ]:
def paginaIndexada(url):

In [31]:
def crawl(paginas, profundidade):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    for i in range(profundidade):
        novas_paginas = set()
        for pagina in paginas:
            http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET', pagina)
            except:
                print('Erro na página ' + pagina)
                continue
            
            sopa = BeautifulSoup(dados_pagina.data, 'html')
            links = sopa.find_all('a')
            for link in links:
                if 'href' in link.attrs:
                    url = urljoin(pagina, str(link.get('href')))
                    if url.find("'") != -1:
                        continue
                    url = url.split('#')[0]
                    if url[0:4] == 'http':
                        novas_paginas.add(url)
                    
            paginas = novas_paginas
    return paginas

In [32]:
teste = crawl(['https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o'], 2)

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Erro na página https://wikimediafoundation.org/wiki/Condições_de_Uso
Erro na página https://be-x-old.wikipedia.org/wiki/%D0%9C%D0%BE%D0%B2%D0%B0_%D0%BF%D1%80%D0%B0%D0%B3%D1%80%D0%B0%D0%BC%D0%B0%D0%B2%D0%B0%D0%BD%D1%8C%D0%BD%D1%8F
Erro na página https://pt.wikipedia.org/w/index.php?title=Linguagem_de_programação&oldid=50743371


In [37]:
for i in teste:
    type(i)

In [33]:
f = open('links.txt', 'w') 

In [38]:
for link in teste:
    f.write('%s' %link)

In [39]:
f.close()

In [8]:
len(teste)

55566

# Pré-processamento dos textos: remoção das tags html

In [3]:
# remover algumas tags
def getTexto(sopa):
    for tag in sopa(['script', 'style']):
        tag.decompose()
    
    return ' '.join(sopa.stripped_strings)

# Pré-processamento dos textos: separacao das palavras

In [17]:
def separaPalavras(texto):
    splitter = re.compile('\\W*')
    stop = nltk.corpus.stopwords.words('portuguese')
    stemmer = nltk.stem.RSLPStemmer()
    lista_palavras = []
    lista = [p for p in splitter.split(texto) if p != '' ] 
    for p in lista:
        if p.lower() not in stop and len(p) > 1:
            lista_palavras.append(stemmer.stem(p).lower())
            
    return lista_palavras

In [19]:
separaPalavras('Este lugar é apavorante')

/home/cloves/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: split() requires a non-empty pattern match.
  


['lug', 'apavor']